# MARKET BASKET ANALYSIS

In [19]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings 
import re 
from mlxtend.frequent_patterns import apriori, association_rules
import networkx as nx
from pyvis.network import Network

In [20]:
df = pd.read_csv(r"C:\Users\takue\Documents\Data Science\Super Store\Sample Superstore - Orders.csv")
df.head()

,Order Priority,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,Product Sub-Category,Product Container,Product Name,Region,State or Province,City,Order Date,Ship Date,Quantity,Order ID
0,Not Specified,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,Labels,Small Box,Avery 49,Central,Illinois,Addison,28/05/2012,30/05/2012,2,88525
1,High,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,Pens & Art Supplies,Wrap Bag,SANFORD Liquid Accent™ Tank-Style Highlighters,West,Washington,Anacortes,07/07/2010,08/07/2010,4,88522
2,Not Specified,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,Paper,Small Box,Xerox 1968,West,Washington,Anacortes,27/07/2011,28/07/2011,7,88523
3,Not Specified,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,"Scissors, Rulers and Trimmers",Small Pack,Acme® Preferred Stainless Steel Scissors,West,Washington,Anacortes,27/07/2011,28/07/2011,7,88523
4,Not Specified,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,Telephones and Communication,Small Box,V70,West,Washington,Anacortes,27/07/2011,27/07/2011,8,88523


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9426 entries, 0 to 9425
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Order Priority        9426 non-null   object 
 1   Unit Price            9426 non-null   float64
 2   Shipping Cost         9426 non-null   float64
 3   Customer ID           9426 non-null   int64  
 4   Customer Name         9426 non-null   object 
 5   Ship Mode             9426 non-null   object 
 6   Customer Segment      9426 non-null   object 
 7   Product Category      9426 non-null   object 
 8   Product Sub-Category  9426 non-null   object 
 9   Product Container     9426 non-null   object 
 10  Product Name          9426 non-null   object 
 11  Region                9426 non-null   object 
 12  State or Province     9426 non-null   object 
 13  City                  9426 non-null   object 
 14  Order Date            9426 non-null   object 
 15  Ship Date            

In [22]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unit Price,9426.0,88.303686,281.540982,0.99,6.4800,20.99,85.99,6783.02
Shipping Cost,9426.0,12.795142,17.181203,0.49,3.1925,6.05,13.99,164.73
Customer ID,9426.0,1738.422236,979.167197,2.00,898.0000,1750.00,2578.75,3403.00
Quantity,9426.0,13.798430,15.107688,1.00,5.0000,10.00,17.00,170.00
Order ID,9426.0,82318.489073,19149.448857,6.00,86737.2500,88344.50,89987.75,91591.00


In [23]:
df.describe(include='object').T

,count,unique,top,freq
Order Priority,9426,6,High,1970
Customer Name,9426,2703,Louis Parrish,27
Ship Mode,9426,3,Regular Air,7036
Customer Segment,9426,4,Corporate,3375
Product Category,9426,3,Office Supplies,5181
Product Sub-Category,9426,17,Paper,1379
Product Container,9426,7,Small Box,4887
Product Name,9426,1263,"Global High-Back Leather Tilter, Burgundy",27
Region,9426,4,Central,2899
State or Province,9426,49,California,1021


#### Model

In [24]:
def choose_region(df, segment,segment_filter, min_support, metric, min_threshold, max_len=None):
    mkt_data = df[['Order ID', segment, 'Product Name']]
    
    # Filter data for the selected region
    data = (mkt_data[mkt_data[segment] == segment_filter]
                .drop(f'{segment}', axis=1)
                .groupby(['Order ID', 'Product Name'])
                .size()
                .unstack()
                .fillna(0)
                .reset_index()
                .set_index('Order ID'))
    
    print(f"Number of rows before removing orders with one products only: {data.shape[0]}")

    # Remove orders with only one unique product
    data = data[data.gt(0).sum(axis=1) > 1]

    print(f"Number of after before removing orders with one products only: {data.shape[0]}")

    # Convert counts to binary format
    data = data.map(lambda x: 1 if x > 0 else 0)

    # Ensure we don't run Apriori on an empty dataset
    if data.empty:
        print("No orders with multiple products found in this region.")
        return None

    # Generate frequent itemsets using Apriori
    model = apriori(data, min_support=min_support, use_colnames=True, max_len=max_len)

    # Generate association rules
    rules = association_rules(model, metric=metric, min_threshold=min_threshold)
    print(f"Number of rules: {rules.shape[0]}" )

    return rules.sort_values(by='lift', ascending=False)


In [25]:
data = choose_region(df, 'Customer Segment' ,'Corporate', 0.002, 'lift', 1.2, max_len=2)
data.head(5)

Number of rows before removing orders with one products only: 2298
Number of after before removing orders with one products only: 802


c:\Users\takue\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Number of rules: 254


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
107,(Bevis Round Conference Table Top & Single Col...,(Round Specialty Laser Printer Labels),0.002494,0.002494,0.002494,1.0,401.0,1.0,0.002488,inf,1.0,1.0,1.0,1.0
226,(Nu-Dell Executive Frame),(Okidata ML184 Turbo Dot Matrix Printers),0.002494,0.002494,0.002494,1.0,401.0,1.0,0.002488,inf,1.0,1.0,1.0,1.0
60,"(Angle-D Binders with Locking Rings, Label Hol...","(SAFCO Mobile Desk Side File, Wire Frame)",0.002494,0.002494,0.002494,1.0,401.0,1.0,0.002488,inf,1.0,1.0,1.0,1.0
106,(Round Specialty Laser Printer Labels),(Bevis Round Conference Table Top & Single Col...,0.002494,0.002494,0.002494,1.0,401.0,1.0,0.002488,inf,1.0,1.0,1.0,1.0
51,(T61),"(Acco Clips to Go™ Binder Clips, 24 Clips in T...",0.002494,0.002494,0.002494,1.0,401.0,1.0,0.002488,inf,1.0,1.0,1.0,1.0


In [26]:
def visualise_baskets(data, saving_name):
    # Create PyVis Network
    G = Network(height="600px", width="100%")

    # Add Nodes & Edges
    for _, row in data.iterrows():
        antecedent = list(row["antecedents"])[0]  
        consequent = list(row["consequents"])[0]  
        lift = row["lift"]
        
        # Add nodes
        G.add_node(antecedent, label=antecedent, color="blue")
        G.add_node(consequent, label=consequent, color="red")

        # Add edge with lift as weight
        G.add_edge(antecedent, consequent, value=lift, title=f"Lift: {lift:.2f}")

    # Save the visualization to an HTML file
    G.save_graph(f"{saving_name}.html")

    print("Network graph saved as f'{saving_name}.html'. Open it in a browser to view.")


In [27]:
visualise_baskets(data, f'corporate_product_associations')

Network graph saved as f'{saving_name}.html'. Open it in a browser to view.
